In [1]:
import os, sys
os.sys.path.append('..')
from recommender.utils import mongodbutils as mdb
from recommender.utils.utils import csv2df
from recommender.recommender.recommenderMF import RecommenderMF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

D:\python\Python35\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\python\Python35\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
D:\python\Python35\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
D:\python\Python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dbconn = ("localhost", 27017)
db_name = "test"
url = "mongodb://{:s}:{:d}/".format(*dbconn)
table = 'hullabalooza'

In [3]:
ds = mdb.DataService(url, db_name)

In [4]:
F = ds.get_features_by_ids(table=table)

Preview some features

In [5]:
F[0:5]

[{'_id': 0,
  'feature': [0.3767674302852362,
   0.37385123601970843,
   -0.3998395679051162,
   -0.41730092507100364,
   -0.3901265469636696,
   0.3763067172444134,
   0.3779431569638814,
   -0.40301334284087026,
   -0.4395536169489178,
   0.3189692701234426]},
 {'_id': 1,
  'feature': [0.36067630426416325,
   0.349863071781424,
   -0.3383256839954945,
   -0.3685303144805748,
   -0.37107527267381013,
   0.3596985794265591,
   0.3659544041750166,
   -0.35863781225661573,
   -0.35750655820359034,
   0.284139029375843]},
 {'_id': 2,
  'feature': [0.2816094941228703,
   0.24535448268675403,
   -0.27126282186466083,
   -0.27897344414972514,
   -0.28973971094724604,
   0.26984837146391405,
   0.27692178797224987,
   -0.27884146318928194,
   -0.2810657038651738,
   0.2106260977794503]},
 {'_id': 3,
  'feature': [0.261376062307955,
   0.2486466097628552,
   -0.2504168147525899,
   -0.2432596413145084,
   -0.2526502585862307,
   0.25156638440425216,
   0.2562532544910389,
   -0.254649725189281

In [210]:
F = pd.DataFrame(F)
Md = pd.DataFrame(ds.get_movie_mds_by_embedding_ids('hullabalooza'))

In [211]:
Md.head()

,_id,embedding_id,r_count,title
0,1,0,215,Toy Story (1995)
1,2,1,110,Jumanji (1995)
2,3,2,52,Grumpier Old Men (1995)
3,4,3,7,Waiting to Exhale (1995)
4,5,4,49,Father of the Bride Part II (1995)


In [212]:
df, user_map, item_map, N, M = csv2df('D:/PycharmProjects/recommender/data/ml-latest-small/ratings.csv',
                                          'movieId', 'userId', 'rating', return_cat_mapping=True)

In [213]:
F = F.merge(Md, left_on='_id', right_on='embedding_id')
F.head()

,_id_x,feature,_id_y,embedding_id,r_count,title
0,0,"[-0.004350134836999435, -0.0020604420440654345...",1,0,215,Toy Story (1995)
1,1,"[-0.0032741171081305063, 0.0018584469784394923...",2,1,110,Jumanji (1995)
2,2,"[0.0021996552978907763, -0.003990608859061313,...",3,2,52,Grumpier Old Men (1995)
3,3,"[-0.0027070414348511693, 0.005564447369848126,...",4,3,7,Waiting to Exhale (1995)
4,4,"[-0.002337712681650084, -0.002467854352299917,...",5,4,49,Father of the Bride Part II (1995)


In [214]:
Fviz = F.loc[F.r_count>20]
Farr = np.array([f for f in F.feature])
Farr.shape

(9724, 20)

In [172]:
from sklearn.manifold import TSNE

In [20]:
tsne = TSNE(metric="cosine")
Farr_emb = tsne.fit_transform(Farr)
Farr_emb.shape

(9724, 2)

In [21]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [22]:
Fviz_emb = Farr_emb[Fviz.index]
trace = go.Scatter(x=Fviz_emb[:,0], y=Fviz_emb[:,1], text=Fviz['title'], mode='markers')

In [23]:
query_ids = np.array([f for f in Fviz.loc[Fviz.title.str.startswith('Harry Potter')].index])
query_emb = Farr_emb[query_ids]
query_titles = Fviz.loc[query_ids]['title']
trace2 = go.Scatter(x=query_emb[:,0], y=query_emb[:,1], text=query_titles, mode='markers')
print(Farr_emb[query_ids])

[[-19.885609  -11.389332 ]
 [-40.020023   34.268604 ]
 [ 45.92649    10.995374 ]
 [ 45.885006    8.576286 ]
 [ 23.131683   15.62371  ]
 [  4.7644844  23.239151 ]
 [-18.561386  -36.60504  ]
 [ 34.118217    6.595133 ]]


In [24]:
fig = go.Figure(data=[trace, trace2])
py.offline.iplot(fig)